© Steadforce GmbH 2020

#0. Get dependencies and the pretrained model

In [1]:
import sys

# Add the root directory to path
sys.path.append("..")


In [2]:
%matplotlib inline

import torch
from lib.model import get_model


use_cuda = torch.cuda.is_available()
device = torch.device('gpu:0') if use_cuda is True else torch.device('cpu')
model = get_model(half=use_cuda, device=device)
print(f'usging device: {device}')

/Users/szymonsadkowski/miniconda3/envs/mim-licence-plate/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


usging device: cpu


In [3]:
from lib.image import load_image_torch, preprocess_img, download_img, img_to_numpy


img_url = 'https://raw.githubusercontent.com/szysad/Licence-Plates-Det-Attacks/main/images/example.jpg'
download_img(img_url, 'plate1.png')
img = load_image_torch('plate1.png').to(device)
img = preprocess_img(img, half=use_cuda)[0]

## Demonstrate prediction bbox

In [4]:
from yolov5.utils.general import non_max_suppression
from lib.model import plot_detections


detections = model(img.unsqueeze(0))
detections = non_max_suppression(detections, conf_thres=model.conf, iou_thres=model.iou)
plot_detections(img_to_numpy(img), detections[0])

## Demonstrate superpixel segmentation

In [5]:
from lib.segmentation import SuperPixler
import matplotlib.pyplot as plt


super_pixler = SuperPixler(img_to_numpy(img), n_segments=30, sigma=0, compactness=60)
plt.imshow(super_pixler.mark_boundaries())

In [6]:
from lib.explanations import calculate_shap, plot_shap_explenations


target_bbox = detections[0][:, :4]
shap_v = calculate_shap(model, super_pixler, target_bbox, nsamples=250, half=use_cuda)

100%|██████████| 1/1 [02:05<00:00, 125.01s/it]


## Demonstate patch contribution heatmap

In [9]:
plot_shap_explenations(img_to_numpy(img), shap_v[0], super_pixler.get_segments(), 0.5)